# Webstruct

- dependencies
    - `lxml` for parsing HTML;
    - `scikit-learn` >= 0.14.

## TODOS
- WebAnnotator: https://www.aclweb.org/anthology/L12-1021/

In [18]:
#!pip install webstruct

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import webstruct
from webstruct import WebAnnotatorLoader

In [3]:
TRAIN_PATH_JSON = "Branchen/data/train.ndjson"
TEST_PATH_JSON = "Branchen/data/test.ndjson"
TRAIN_PATH_CSV = "Branchen/data/train.csv"
TEST_PATH_CSV = "Branchen/data/test.csv"

## Load csvs and create test html

In [7]:
%%time
train = pd.read_csv(TRAIN_PATH_CSV)
train.head(2)

CPU times: user 29.4 s, sys: 10.5 s, total: 39.9 s
Wall time: 57.7 s


,text,html,industry,country,industry_name
0,Home | NETZkultur GmbH\n\nZum Inhalt wechseln\...,"<!DOCTYPE html>\n<html lang=""de-DE"">\n<head>\n...",4,DE,Computer Software
1,"\n\nNXP Semiconductors | Automotive, Security,...",<!DOCTYPE html>\n<html>\n<head>\n\t<title>NXP ...,7,NaN,Semiconductors


In [24]:
testhtml = train.iloc[0]["html"]
with open("train/test.html", "w+") as f:
    f.write(testhtml)

TODO: all HTMLs to train

## ...

In [26]:
loader = WebAnnotatorLoader()  
loader.load("train/test.html")

<Element html at 0x7f821d9b8bf0>

In [25]:
trees = webstruct.load_trees("train/*.html", WebAnnotatorLoader())

In [27]:
html_tokenizer = webstruct.HtmlTokenizer()
X, y = html_tokenizer.tokenize(trees)

TODO: https://webstruct.readthedocs.io/en/latest/tutorial.html